### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v.1.0.21'

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.21'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip:
    - python-dotenv
    - azureml-sdk[notebooks,automl,explain] 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

FROM continuumio/miniconda3:4.5.12
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# basic system packages, plus gcc
RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 apt-transport-https ca-certificates curl gnupg git && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH         

# Install Azure CLI
ENV AZ_CLI_REPO=stretch
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli        
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.21


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + image_version_string


docker_file_path
docker_image_name
docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.21'

'georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.21'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001  282 Apr  3 04:19 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 1247 Apr  3 04:19 dockerfile_aml-sdk_docker_image_sdk.v.1.0.21


In [9]:
!echo docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache
!     docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache

docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.21 -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.21 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache
Sending build context to Docker daemon  4.096kB
Step 1/14 : FROM continuumio/miniconda3:4.5.12
 ---> ae46c364060f
Step 2/14 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in c00bd806d302
Removing intermediate container c00bd806d302
 ---> 0fa1329081ba
Step 3/14 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2 apt-transport-https ca-certificates curl gnupg git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in b6b55386b409
Get:1 http://security.debian.org/debian-security stretch/updates InRelease [94.3 kB]
Ign:2 http://deb.debian.org/debian stretch InRelease
Get:3 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Get:4 http://deb.debian

Selecting previously unselected package pinentry-curses.
Preparing to unpack .../02-pinentry-curses_1.0.0-2_amd64.deb ...
Unpacking pinentry-curses (1.0.0-2) ...
Selecting previously unselected package libnpth0:amd64.
Preparing to unpack .../03-libnpth0_1.3-1_amd64.deb ...
Unpacking libnpth0:amd64 (1.3-1) ...
Selecting previously unselected package readline-common.
Preparing to unpack .../04-readline-common_7.0-3_all.deb ...
Unpacking readline-common (7.0-3) ...
Selecting previously unselected package libreadline7:amd64.
Preparing to unpack .../05-libreadline7_7.0-3_amd64.deb ...
Unpacking libreadline7:amd64 (7.0-3) ...
Selecting previously unselected package gnupg-agent.
Preparing to unpack .../06-gnupg-agent_2.1.18-8~deb9u4_amd64.deb ...
Unpacking gnupg-agent (2.1.18-8~deb9u4) ...
Selecting previously unselected package libksba8:amd64.
Preparing to unpack .../07-libksba8_1.3.5-2_amd64.deb ...
Unpacking libksba8:amd64 (1.3.5-2) ...
Selecting previously unselected package libsqlite3-0:

 ---> Running in 089b4cf7899e
Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... + /opt/conda/envs/aml-sdk-conda-env/bin/python -m nb_conda_kernels.install --enable
Enabling nb_conda_kernels...
Status: enabled

+ /opt/conda/envs/aml-sdk-conda-env/bin/jupyter-nbextension enable nb_conda --py --sys-prefix
Enabling notebook extension nb_conda/main...
      - Validating: OK
Enabling tree extension nb_conda/tree...
      - Validating: OK
+ /opt/conda/envs/aml-sdk-conda-env/bin/jupyter-serverextension enable nb_conda --py --sys-prefix
Enabling: nb_conda
- Writing config: /opt/conda/envs/aml-sdk-conda-env/etc/jupyter
    - Validating...
      nb_conda 2.2.1 OK

done


  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Stored in directory: /root/.cache/pip/wheels/bd/de/48/90f1e21c58db8ab2e723ad80c46db9edf92a791c367166ed8f
  Stored in directory: /root/.cache/pip/wheels/bf/26/bd/912db1314f1cef0171d9b7f128dd01e8b8c92ed8d0062e632d
  Stored in directory: /root/.cache/pip/wheels/ee/30/5c/3a3b5e1386c8db9a3be5f5c3933644ae0533c1351c6a8eb4b5
  Stored in directory: /root/.cache/pip/wheels/e8/74/51/42c2d41c02bdc6f0e604476b7e4293b8c98d0bcbfa1dff78c8
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
  Stored in directory: /root/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
Successfully built pathspec pyyaml di

  Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16.2:
      Successfully uninstalled numpy-1.16.2
  Found existing installation: scipy 1.2.1
    Uninstalling scipy-1.2.1:
      Successfully uninstalled scipy-1.2.1
  Found existing installation: wheel 0.33.1
    Uninstalling wheel-0.33.1:
      Successfully uninstalled wheel-0.33.1
Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
jedi-0.13.3-py36_0.tar.bz2                   236 KB
pickleshare-0.7.5-py36_0.tar.bz2              13 KB
setuptools-40.8.0-py36_0.tar.bz2             647 KB
pygments-2.3.1-py36_0.tar.bz2                1.4 MB
notebook-5.7.7-py36_0.tar.bz2                7.3 MB
six-1.12.0-py36_0.tar.bz2                     22 KB
readline-7.0-h7b6447c_5.tar.bz2              392 KB
jupyter_core-4.4.0-py36_0.tar.bz2             63 KB
nb_conda-2.2.1-py36_0.tar.bz2                 33 KB
ptyprocess-0.6.0-py36_0.tar.bz2               23 KB
libgcc-ng-8.2.0-hdf63c60_

Removing intermediate container 089b4cf7899e
 ---> 3ae2e120c0cd
Step 8/14 : ENV CONDA_AUTO_UPDATE_CONDA=false
 ---> Running in 6543042eb9e8
Removing intermediate container 6543042eb9e8
 ---> fa5ddfe31984
Step 9/14 : ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
 ---> Running in b07d54c8603b
Removing intermediate container b07d54c8603b
 ---> 0195e29e0cf6
Step 10/14 : ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
 ---> Running in f13f989ef861
Removing intermediate container f13f989ef861
 ---> f0bcb27f74c4
Step 11/14 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in ce45e45fcf41
Removing intermediate container ce45e45fcf41
 ---> c064edb9219d
Step 12/14 : ENV AZ_CLI_REPO=stretch
 ---> Running in fd4b03a552e1
Removing intermediate container fd4b03a552e1
 ---> 26ea504f52c9
Step 13/14 : RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" |     tee /etc/apt/sources.list.d/azure-cli.list &&     curl -L https://packages.microsoft.com/keys/

In [10]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir'

total 8
-rw-r--r-- 1 root root 4720 Apr  3 04:17 exported_conda_env.yml


In [11]:
# export the resulted conda env
!docker run  \
        -v $crt_local_dir:/workspace:rw \
        $docker_image_name \
        /bin/bash -c "conda env export > /workspace/exported_conda_env.yml"

In [12]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [13]:
# !docker login -u=$docker_login -p=
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

8e982561: Preparing 
930e30af: Preparing 
07ca8952: Preparing 
ce217f1b: Preparing 
6384d46a: Preparing 
8fbc5764: Preparing 
828fac1a: Preparing 
ae060f2d: Preparing 


930e30af: Pushing  1.614GB/2.02GBBAPushing  327.3MB/2.02GBPushing  1.525GB/2.02GB

930e30af: Pushed   2.068GB/2.02GBPushing  1.884GB/2.02GBsdk.v.1.0.21: digest: sha256:84d2d1df41b97380bd5c6463d2908d1f234c90115bb8d8f42a39bddec40c2a1b size: 2214


### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [14]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.21 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [15]:
!echo docker run  $docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

docker run georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.21 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.21


#### Do the same in a  notebook

In [16]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [17]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir/test_aml_sdk_docker_image.ipynb


In [18]:
!ls -l $crt_local_dir

total 12
-rw-r--r-- 1 root       root       4720 Apr  3 04:27 exported_conda_env.yml
-rw-rw-r-- 1 loginvm001 loginvm001  982 Apr  3 04:29 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [19]:
!docker run -v $crt_local_dir:/workspace:rw $docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [20]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.21\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


In [21]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 363038 bytes to createAMLSDKDocker.html
